In [ ]:
import pandas as pd
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
import random

import sklearn as sk

# Preparation

## Load Data

In [ ]:
# read chord data

data_chords = pd.read_csv('chord_list.tsv', sep='\t')

data_chords

In [ ]:
sequences_chords = [np.array(piece["chord"]) for name, piece in data_chords.groupby("id")]
sequences_chords[0]

In [ ]:
sns.distplot([len(s) for s in sequences_chords])

## Code for Computing Mutual Information Statistics

See "Critical Behaviour in Physics and Probabilistic Formal Languages" (Lin & Tenmark, 2017), Appendix D,
and "Parallels in the sequential organization of birdsong and human speech" (Sainburg et al., 2019).

In [ ]:
# estimate the entropy of a distribution from samples
# expects a list or numpy array of samples
def est_entropy(samples):
    N = len(samples)
    counter = Counter(samples)
    counts = np.fromiter(Counter(samples).values(), dtype=int)
    return np.log2(N) - (sum(counts * scipy.special.psi(counts)) / N)
    #return np.log2(N) - ((sum([counts * scipy.special.psi(counts) for (key, counts) in counter.items()])) / N)

# estimate the mutual information between two variable from sample pairs
# expects an array with two columns
def est_mi(pairs):
    X = pairs[:,0]
    Y = pairs[:,1]
    pairs_tup = list(map(tuple,pairs))
    sX = est_entropy(X)
    sY = est_entropy(Y)
    sP = est_entropy(pairs_tup)
    return sX + sY - sP

def est_mi_corr(pairs, shuffled_pairs):
    return est_mi(pairs) - est_mi(shuffled_pairs)

In [ ]:
def seq_pairs(sequence, distance):
    return np.column_stack([sequence[:-distance], sequence[distance:]])

def seqs_pairs(sequences, distance):
    #print(distance)
    return np.concatenate([seq_pairs(seq, distance) for seq in sequences])

In [ ]:
def show_progress(dist):
    print(dist)
    return dist

def collect_mi(seqs, maxdist=10):
    maxdist = min(maxdist+1, max(map(len, seqs)))
    shuffled = [np.random.permutation(s) for s in seqs]
    values = [(show_progress(dist), est_mi_corr(seqs_pairs(seqs, dist), seqs_pairs(shuffled, dist))) for dist in range(1,maxdist)]
    return pd.DataFrame.from_records(values, columns=["dist", "mi"])

# Exercise 1: Plot Mutual Information

In this exercise we want to look at some information theoretic properties of the chord sequences we loaded above.
The [mutual information](https://en.wikipedia.org/wiki/Mutual_information) between two random variables $A$ and $B$ measures how much we learn about $B$ by observing $A$ and vice versa.
In a chord sequence, for example, we can ask how much knowing the first chord tells us about the third cord.
In this case we want to look at pairs of chords with a certain distance, e.g., how much do we learn from a chord about the chord that comes 2, 3, or 4 steps later.
With increasing distance, this mutual information is probably going to decrease, but we want to look at how exactly this decay looks like.

Compute the mutual information for all chord distances and plot them.
Use the function `collect_mi`, which takes a list and an optional maximum distance, and returns a dataframe with the mutual information depending on the distance.

# Exercise 2: Markov Model

We now want to see, how well a markov model can model the structure of the chord sequences.
For this exercise you can reuse your solution from two weeks ago.

1. Use the chord sequences to train a 1st-order markov model.
1. Use the model to generate new sequences of length 200.
1. Compute and plot the mutual information statistics on these sequences. What do you observe?

## Exercise 3 (optional): Modelling the Decay of Mutual Information

In the last exercise we saw how the decay of information changes when reconstructing the sequences from a markov model. In this exercise we want to see if the change is caused by the model not being "big" enough (a higher-order model might perform better, after all), or if the model class is not correct in the first place.

In a Markov model we expect an exponential decay of mutual information with increasing distance.
The intuitive reason for that is that if we go more than one step, we multiply the transition probabilities.
For $x$ steps, we therefore take the probability to the $x$th power.
The same is true for a markov model of higher order, except that the curve is less steep.
We can therefore model the decay as $MI(x) = a e^{xb} + c$, where $x$ is the distance.

Another decay function is a power law, where the distance is not in the exponent but is instead taken to a fixed negative power: $MI(x) = a x^b + c$.
This function decays asymtotically slower than the exponential and cannot be generated by a Markov model.

Try to fit both functions to the mutual information you observed in the original sequences. You can use the function [`scipy.optimize.curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) together with these function definitions:

In [ ]:
exponential = lambda x, a, b, c: a * (np.e ** (-x*b)) + c
powerlaw = lambda x, a, b, c: a * (x ** b) + c

You will get a list of estimates for the parameters `a`, `b`, and `c` in each case.
Use these to

1. compute the estimated MI values for each distance,
1. compute the sum of squared errors between the estimates and the actual data, and
1. plot observed data together with the two fitted curves.

Which model explains the observed data better?